<a href="https://colab.research.google.com/github/KeqingW44448/api/blob/main/RSM8421/5_1_Introduction_to_Word_Vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Vectors in Python with gensim

This notebook shows an example of how to use word vectors in Python with gensim.

## Load Libraries

We're using a new library called `gensim`.  It's a great library for modeling text and comes with pre-trained models that you can easily use in other contexts.

In [ ]:
%matplotlib inline
import numpy as np
!pip install gensim
import gensim
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

### Cleaning Our Corpus

(corpus is another name for the set of documents under consideration)

In [ ]:
documents = [
    "Perfect is the enemy of good.",
    "I'm still learning.",
    "Life is a journey, not a destination.",
    "Learning is not attained by chance, it must be sought for with ardor and attended to with diligence.",
    "Yesterday I was clever, so I changed the world. Today I am wise, so I am changing myself.",
    "Be curious, not judgmental.",
    "You don't have to be great to start, but you have to start to be great.,"
    "Be stubborn about your goals and flexible about your methods.",
    "Nothing will work unless you do.",
    "Never give up on a dream just because of the time it will take to accomplish it. The time will pass anyway.",
    "Anyone who stops learning is old, whether at twenty or eighty.",
    "Tell me and I forget. Teach me and I remember. Involve me and I learn.",
    "Change is the end result of all true learning.",
    "Live as if you were to die tomorrow. Learn as if you were to live forever.",
    "A learning curve is essential to growth.",
]

# remove common words and tokenize
stop_words = set('for a of the and to in'.split())
texts = [[word for word in document.lower().replace("'", "").replace(".", "").split() if word not in stop_words]
         for document in documents]

## remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
texts

### Creating our Word2Vec Model

`gensim` makes it easy to train a Word2Vec model.  All training requires is passing in the corpus.

In [ ]:
model = gensim.models.Word2Vec(texts, vector_size=10, window=2, min_count=1)
model

In [ ]:
model.wv['live']

And we can find the most similar words too.  Obviously, our dataset is too small and we won't find anything too interesting.

In [ ]:
model.wv.most_similar('live')

### Loading an existing corpus

We can load some existing text and train a model on it. In this case, we're going to use `text8`, which is a small subset of Wikipedia (31MB). See: https://github.com/RaRe-Technologies/gensim-data

In [ ]:
import gensim.downloader as api

# This could be slow to download...
corpus = api.load("text8")
corpus

In [ ]:
# Using small numbers here, probably want to use a bigger corpus, bigger dimensions, and more iterations.
model = gensim.models.Word2Vec(corpus, vector_size=10, window=2, epochs=5, min_count=1)
model

We can get slightly better results (but we really should be using a much bigger corpus)

In [ ]:
model.wv.most_similar("queen")

In [ ]:
model.wv['queen']

### Loading a pre-trained model

We can also use Gensim to automatically download and load a pre-trained model, or load it from disk. Since the pre-trained model has much more data, the vectors encode some semantic meaning.

In [ ]:
# Glove is another word embedding that uses a slightly different technique than word2vec,
# however, it has the same properties and API
model = api.load("glove-wiki-gigaword-50")
model

In [ ]:
model.most_similar('queen')

Alternatively, we can load the same model directly from disk (the previous calls cache the files in `~/gensim-data/`.

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('~/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz')
model

In [ ]:
model.most_similar('queen')

In [ ]:
model['queen']

### Visualizing Words with T-SNE

As we saw in the slides, we can visualize the distance between words using T-SNE.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# Gather a listing of random words
words = ['queen', 'princess', 'dog', 'king', 'cat', 'obama', 'clinton', 'president', 'math', 'brian']

vecs = np.array([model[w] for w in words])
vecs

In [ ]:
vecs_tsne = TSNE(n_components=2, perplexity=3).fit_transform(vecs)
vecs_tsne

In [ ]:
ax = sns.scatterplot(x=vecs_tsne[:, 0], y=vecs_tsne[:, 1], s=100)
sns.set(font_scale=1.5)
[ax.text(p[0], p[1]+10, word, color='black') for word, p in zip(words, vecs_tsne)]
pass